In [52]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
import torchvision.models as models
import torch.optim
from torch.utils.data import DataLoader, random_split
import torchmetrics
from torchmetrics.classification import Accuracy

from sklearn.model_selection import KFold

In [23]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [24]:
train_transform= transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform= transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform= transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [40]:
!ls /kaggle/input/kermany2018/oct2017/

 __MACOSX  'OCT2017 '


In [41]:
train_dir = "/kaggle/input/kermany2018/oct2017/OCT2017 /train"
val_dir = "/kaggle/input/kermany2018/oct2017/OCT2017 /val"
test_dir = "/kaggle/input/kermany2018/oct2017/OCT2017 /test"


train_data= datasets.ImageFolder(root= train_dir, transform= train_transform)
val_data= datasets.ImageFolder(root= val_dir, transform= val_transform)
test_data= datasets.ImageFolder(root= test_dir, transform= test_transform)

In [50]:
train_loader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=128, shuffle=False, num_workers=2)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False, num_workers=2)


In [44]:
model= models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 4) # 4 classification classes

In [46]:
model.fc.in_features

2048

In [ ]:
model.to(device)

In [49]:
loss_function= nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(), lr=0.001)
scheduler= torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience= 3)

In [51]:
def train(dataloader, model, loss_function, optimizer):
    model.train()
    total_loss=0

    for batch, (image, label) in enumerate(dataloader):
        image= image.to(device)
        label= label.to(device)
        prediction= model(image)
        loss= loss_function(prediction, label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss+= loss

    avg_loss= total_loss/len(dataloader)
    print(f"Training Average Loss: {avg_loss:.4f}")

In [59]:
val_accuracy= Accuracy(task="multiclass", num_classes=4).to(device)

def validate(dataloader, model, loss_function):
    model.eval()
    total_loss=0
    val_accuracy.reset()

    with torch.no_grad():
        for image, label in dataloader:
            image= image.to(device)
            label= label.to(device)
            prediction= model(image)
            loss= loss_function(prediction, label)
            total_loss+= loss

            val_accuracy.update(prediction, label)

        avg_loss= total_loss/ len(dataloader)
        accuracy= val_accuracy.compute()*100
    print(f"Validation Loss: {avg_loss:.4f}")
    print(f"Validation Accuracy: {accuracy:.2f}")

    return accuracy

In [60]:
test_accuracy= Accuracy(task="multiclass", num_classes=4).to(device)

def test(dataloader, model, loss_function):
    model.eval()
    total_loss=0
    val_accuracy.reset()

    with torch.no_grad():
        for image, label in dataloader:
            image= image.to(device)
            label= label.to(device)
            prediction= model(image)
            loss= loss_function(prediction, label)
            total_loss+= loss

            test_accuracy.update(prediction, label)

        avg_loss= total_loss/ len(dataloader)
        accuracy= test_accuracy.compute()*100
    print(f"Test Loss: {avg_loss:.4f}")
    print(f"Test Accuracy: {accuracy:.2f}")

    return accuracy

In [61]:
epochs= 50
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train(train_loader, model, loss_function, optimizer)
    val_acc= validate(val_loader, model, loss_function)
    scheduler.step(val_acc)

test(test_loader, model, loss_function)


Epoch 1/50
Training Average Loss: 0.1376
Validation Loss: 0.0258
Validation Accuracy: 100.00

Epoch 2/50
Training Average Loss: 0.1316
Validation Loss: 0.0671
Validation Accuracy: 96.88

Epoch 3/50
Training Average Loss: 0.1224
Validation Loss: 0.0708
Validation Accuracy: 100.00

Epoch 4/50
Training Average Loss: 0.1159
Validation Loss: 0.0904
Validation Accuracy: 93.75

Epoch 5/50


KeyboardInterrupt: 